In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
from torchvision import datasets, transforms
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F

In [2]:
# image transformation
transform = transforms.Compose([
    transforms.Resize((128, 128)),   # Resize all images
    transforms.ToTensor(),           # Convert to PyTorch tensor
    transforms.Normalize([0.5]*3, [0.5]*3)  # Normalize RGB channels
])

In [3]:
# load training and test data
train_dataset = datasets.ImageFolder("/kaggle/input/brain-tumor-mri-dataset/Training", transform=transform)
test_dataset = datasets.ImageFolder("/kaggle/input/brain-tumor-mri-dataset/Testing", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
# confirm folders
import os
# print(os.listdir("/kaggle/input/brain-tumor-mri-dataset/Training"))
print(os.listdir("/kaggle/input/brain-tumor-mri-dataset/Testing"))

['pituitary', 'notumor', 'meningioma', 'glioma']


In [5]:

# build CNN model
class BrainTumorCNN(nn.Module):
    def __init__(self):
        super(BrainTumorCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 4)  # 4 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (3x128x128) -> (32x64x64)
        x = self.pool(F.relu(self.conv2(x)))  # (32x64x64) -> (64x32x32)
        x = x.view(-1, 64 * 32 * 32)          # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BrainTumorCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
for epoch in range(15):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")

Epoch 1, Loss: 118.2725
Epoch 2, Loss: 51.4421
Epoch 3, Loss: 29.1371
Epoch 4, Loss: 16.7730
Epoch 5, Loss: 8.4638
Epoch 6, Loss: 3.8824
Epoch 7, Loss: 1.2087
Epoch 8, Loss: 1.7727
Epoch 9, Loss: 2.0150
Epoch 10, Loss: 1.1874
Epoch 11, Loss: 12.8802
Epoch 12, Loss: 2.5470
Epoch 13, Loss: 1.5327
Epoch 14, Loss: 0.8449
Epoch 15, Loss: 1.7574


In [8]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 96.11%
